In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('titles.csv')

In [ ]:
df.head(5)

In [ ]:
df.columns

In [ ]:
from sqlite3 import connect

In [ ]:
conn = connect(':memory:')
df_sql = df.to_sql('test_data', conn)

In [ ]:
# Want to visualize somehow - should be a way to select rows and columns to create a display. 
# what if we selected only movies, made in united states , genre comedy, and compared it to the popularity over time in a line graph


In [ ]:
df.head(5)
df_selected = df[['type', 'genres', 'production_countries','tmdb_popularity']]
df_selected.head(5)

In [ ]:
# Define filtering criteria
movie_criteria = (df['type'] == 'MOVIE') & \
                 (df['genres'].apply(lambda x: 'comedy' in x)) & \
                 (df['production_countries'].apply(lambda x: 'US' in x))

# Apply filters and create a new DataFrame
filtered_df = df[movie_criteria][['title','type', 'genres', 'production_countries', 'tmdb_popularity','release_year']]

# Sort the new DataFrame by tmdb_popularity score in descending order
filtered_df = filtered_df.sort_values(by='tmdb_popularity', ascending=False)

# Reset index of the new DataFrame
filtered_df = filtered_df.reset_index(drop=True)

print(filtered_df)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x_axis = filtered_df['release_year']
y_axis = filtered_df['tmdb_popularity']

In [ ]:

# plt.plot(x_axis, y_axis)
# plt.show

# example of how you could do a line plot , does not work well here



In [ ]:
plt.bar(x_axis, y_axis)
plt.title('Movie tmdb popularity over time')
plt.show()

In [ ]:
# no one cares about old comedy movies ? there is one movie before 1960 and not another until around 1978?
# you can see the spike in 2020 , covid related increased movie watching? would be interesting to
# compare other genres to see if it has similar results 

# Very important! Use of filter on dataframe, with example of how you can filter columns of dataframe
# useing lambda functions to select yes/no


In [ ]:
second_criteria = (df['type'] == 'SHOW') & \
                 (df['genres'].apply(lambda x: 'comedy' in x)) & \
                 (df['production_countries'].apply(lambda x: 'US' in x))

# Apply filters and create a new DataFrame
filtered_df_second = df[second_criteria][['title','type', 'genres', 'production_countries', 'tmdb_popularity','release_year']]

# Sort the new DataFrame by tmdb_popularity score in descending order
filtered_df_second = filtered_df_second.sort_values(by='tmdb_popularity', ascending=False)

# Reset index of the new DataFrame
filtered_df_second = filtered_df_second.reset_index(drop=True)

In [ ]:
x_axis = filtered_df_second ['release_year']
y_axis = filtered_df_second ['tmdb_popularity']
plt.bar(x_axis, y_axis)
plt.title('Comedy Shows in US tmbd rating')
plt.show

In [ ]:
# interesting insights here are comedy shows only started coming out 30 years ago, and there was only 1 that was popular
# then you can see there have been 3 huge hits before 2015 while everthing else is maybe 1/3 as popular
# you could now go ahead and do some sql analysis and find what each of these high rated comedy shows are called
# rather than being a highly rated on tmbp popularity

In [ ]:
conn = connect(':memory:')
df_sql = filtered_df_second.to_sql('test_data', conn)

In [ ]:
pd.read_sql('SELECT * FROM test_data LIMIT 10', conn).reset_index(inplace=True)

In [ ]:
pd.read_sql('SELECT * FROM test_data WHERE release_year < 2015 order by tmdb_popularity DESC LIMIT 20 ', conn)

In [ ]:
filtered_df.to_sql('mov_data', conn)



In [ ]:
#pd.read_sql('SELECT * FROM mov_data LIMIT 10', conn)
pd.read_sql('SELECT * FROM mov_data WHERE release_year < 2015 order by tmdb_popularity DESC LIMIT 20 ', conn)

In [ ]:
# ideas so far would be to further apply filters to already given data to see any patterns ie comedy and romance
# and comedy and fantasy or diffferences between the score and age certification, or how runtime plays a role 
# in overall success or failure

# another key to note here is the unyielding prevelance that sql queries are going to be insanely helpful in 
# obtaining the data your looking for. 
# Example - grouping all release years together to see how many movies were released in a certain year 
# you could group and average the tmdb scores based upon their generes to see which has the highest
# overall score. ie comedy romance - 34 action thriller - 56


# i think for today it would be fun to implment machine learning to learn which variables play the biggest
# role in determining the tmbd score, for the end goal to be to determine what the absolute best combonation
# would be to obtain the BEST POSSIBLE tmb score. You just have to find what varibles you want to test against
# the imdb or tmbd score. 

In [ ]:
df.columns

In [ ]:
df

Your going to need to clean the data somehow, there is going to be errors when you are comapring runtime and seasons with movies and shows while one or the other will have no value. also need to make sure that what you are comparing has values! such as the score or popularity, if they are missing that will skew data heavily


In [ ]:
df.isnull().values.sum()

In [ ]:
# first thing you want to do is create 2 seperate dataframes one contains movies and the other contains shows 
# while also holding all the other variables you wish to test with as X and the tmdb score as Y


In [ ]:
df.columns

# df for shows

In [119]:

df_shows = df[['type','release_year','age_certification', 'runtime', 'genres', 'production_countries','seasons', 'imdb_score', 'imdb_votes', 'tmdb_popularity']]
# create filter to target just shows 


In [131]:
criteria = (df['type'] == 'SHOW')
df_shows = df[criteria][['type','release_year','age_certification', 'runtime', 'genres', 'production_countries','seasons', 'imdb_score', 'imdb_votes', 'tmdb_popularity']]
df_shows

,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score,imdb_votes,tmdb_popularity
0,SHOW,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,0.600
5,SHOW,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8,73424.0,17.617
26,SHOW,1972,TV-MA,43,['comedy'],[],1.0,8.1,2151.0,1.487
35,SHOW,1989,TV-PG,24,['comedy'],['US'],9.0,8.9,308824.0,130.213
44,SHOW,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9,34115.0,50.267
...,...,...,...,...,...,...,...,...,...,...
5828,SHOW,2021,NaN,39,"['documentation', 'music', 'reality']",['BR'],1.0,5.3,45.0,1.002
5830,SHOW,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,8.2,5.0,2.334
5831,SHOW,2021,TV-MA,37,"['drama', 'romance']",['IN'],1.0,5.1,738.0,1.400
5833,SHOW,2021,TV-14,24,['comedy'],['NG'],1.0,NaN,NaN,0.840


In [77]:
df_shows.isnull().sum()

type                      0
release_year              0
age_certification       255
runtime                   0
genres                    0
production_countries      0
seasons                   0
imdb_score              167
imdb_votes              167
tmdb_popularity          29
dtype: int64

In [126]:
df_shows = df_shows.dropna()
df_shows.isnull().sum()

id                      0
title                   0
type                    0
description             0
release_year            0
age_certification       0
runtime                 0
genres                  0
production_countries    0
seasons                 0
imdb_id                 0
imdb_score              0
imdb_votes              0
tmdb_popularity         0
tmdb_score              0
dtype: int64

In [87]:
df_shows.columns

Index(['type', 'release_year', 'age_certification', 'runtime', 'genres',
       'production_countries', 'seasons', 'imdb_score', 'imdb_votes',
       'tmdb_popularity'],
      dtype='object')

In [95]:
df_shows

,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score,imdb_votes,tmdb_popularity
5,SHOW,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8,73424.0,17.617
26,SHOW,1972,TV-MA,43,['comedy'],[],1.0,8.1,2151.0,1.487
35,SHOW,1989,TV-PG,24,['comedy'],['US'],9.0,8.9,308824.0,130.213
44,SHOW,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9,34115.0,50.267
45,SHOW,1984,TV-Y,10,"['animation', 'family', 'comedy', 'fantasy', '...",['GB'],24.0,6.5,5104.0,42.196
...,...,...,...,...,...,...,...,...,...,...
5785,SHOW,2022,TV-MA,16,['comedy'],['ZA'],1.0,4.9,45.0,1.629
5801,SHOW,2021,TV-14,47,"['documentation', 'sport']",['US'],1.0,6.5,162.0,2.523
5819,SHOW,2021,TV-MA,49,"['documentation', 'drama']",['IN'],1.0,8.3,2346.0,1.493
5830,SHOW,2021,TV-14,60,"['drama', 'comedy']",['TH'],1.0,8.2,5.0,2.334


In [ ]:
# now i think it becomes appropriate to start splitting and testing the data ! You have the rows that 
# you want to test along with the varible your seeking to find


In [88]:
X = df_shows[['type', 'release_year', 'age_certification', 'runtime', 'genres', 'production_countries',
        'seasons', 'imdb_votes', 'tmdb_popularity']] 
Y = df_shows[['imdb_score']]

In [89]:
X.shape


(1710, 9)

In [90]:
Y.shape

(1710, 1)

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.linear_model import LinearRegression

In [92]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [93]:
clf = RandomForestClassifier()

In [96]:


from sklearn.preprocessing import OneHotEncoder

In [127]:
df_shows 

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
5,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306
26,ts45948,Monty Python's Fliegender Zirkus,SHOW,Monty Python's Fliegender Zirkus consisted of ...,1972,TV-MA,43,['comedy'],[],1.0,tt0202477,8.1,2151.0,1.487,7.000
35,ts20681,Seinfeld,SHOW,A stand-up comedian and his three offbeat frie...,1989,TV-PG,24,['comedy'],['US'],9.0,tt0098904,8.9,308824.0,130.213,8.301
44,ts22082,Knight Rider,SHOW,"Michael Long, an undercover police officer, is...",1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,tt0083437,6.9,34115.0,50.267,7.500
45,ts21715,Thomas & Friends,SHOW,Thomas & Friends is a British children's telev...,1984,TV-Y,10,"['animation', 'family', 'comedy', 'fantasy', '...",['GB'],24.0,tt0086815,6.5,5104.0,42.196,6.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784,ts285471,The Wedding Coach,SHOW,Former bride and forever comedian Jamie Lee sh...,2021,TV-MA,33,"['comedy', 'reality']",['US'],1.0,tt14210424,5.0,87.0,1.525,10.000
5785,ts325991,Only Jokes Allowed,SHOW,Six of South Africa's top comedians take cente...,2022,TV-MA,16,['comedy'],['ZA'],1.0,tt17052148,4.9,45.0,1.629,10.000
5801,ts270616,We Are: The Brooklyn Saints,SHOW,A Brooklyn youth football program and its self...,2021,TV-14,47,"['documentation', 'sport']",['US'],1.0,tt13656220,6.5,162.0,2.523,10.000
5819,ts287729,Alma Matters: Inside the IIT Dream,SHOW,"In a ""nation of middle-class"" the IIT dream in...",2021,TV-MA,49,"['documentation', 'drama']",['IN'],1.0,tt14512938,8.3,2346.0,1.493,9.000


In [124]:

df_shows = df[df['production_countries'].apply(lambda x: len(x) > 0)] 


In [109]:
df_shows_encoded

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,type_SHOW,age_certification_TV-14,age_certification_TV-G,age_certification_TV-MA,...,"production_countries_['US', 'IN']","production_countries_['US', 'IT']","production_countries_['US', 'KR']","production_countries_['US', 'MX']","production_countries_['US', 'NG']",production_countries_['US'],production_countries_['XX'],"production_countries_['ZA', 'US']",production_countries_['ZA'],production_countries_[]
5,1969.0,30.0,4.0,8.8,73424.0,17.617,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26,1972.0,43.0,1.0,8.1,2151.0,1.487,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
35,1989.0,24.0,9.0,8.9,308824.0,130.213,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,1982.0,51.0,4.0,6.9,34115.0,50.267,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
45,1984.0,10.0,24.0,6.5,5104.0,42.196,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1703,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1704,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1705,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [110]:
df_shows_encoded.isnull().sum().sum()

1110438

In [129]:
df_shows.isnull().sum().sum()

0

In [133]:
df_shows.columns

Index(['type', 'release_year', 'age_certification', 'runtime', 'genres',
       'production_countries', 'seasons', 'imdb_score', 'imdb_votes',
       'tmdb_popularity'],
      dtype='object')

In [143]:
df_dum_prod = pd.get_dummies(df_shows['production_countries'])
df_dum_prod

,['AE'],"['AR', 'ES']","['AR', 'PE']",['AR'],"['AT', 'DE']",['AT'],"['AU', 'FR', 'DE']","['AU', 'GB']","['AU', 'US']",['AU'],...,"['US', 'IT']","['US', 'JP']","['US', 'KR']","['US', 'MX']","['US', 'NG']",['US'],['XX'],"['ZA', 'US']",['ZA'],[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
clean = df_shows.drop(columns =['type', 'production_countries'])
clean

,release_year,age_certification,runtime,genres,seasons,imdb_score,imdb_votes,tmdb_popularity
0,1945,TV-MA,51,['documentation'],1.0,NaN,NaN,0.600
5,1969,TV-14,30,"['comedy', 'european']",4.0,8.8,73424.0,17.617
26,1972,TV-MA,43,['comedy'],1.0,8.1,2151.0,1.487
35,1989,TV-PG,24,['comedy'],9.0,8.9,308824.0,130.213
44,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",4.0,6.9,34115.0,50.267
...,...,...,...,...,...,...,...,...
5828,2021,NaN,39,"['documentation', 'music', 'reality']",1.0,5.3,45.0,1.002
5830,2021,TV-14,60,"['drama', 'comedy']",1.0,8.2,5.0,2.334
5831,2021,TV-MA,37,"['drama', 'romance']",1.0,5.1,738.0,1.400
5833,2021,TV-14,24,['comedy'],1.0,NaN,NaN,0.840


In [144]:
df_dum_genre = pd.get_dummies(clean['genres'])

In [145]:
df_dum_genre

,"['action', 'animation', 'comedy', 'drama', 'family', 'fantasy', 'music', 'romance']","['action', 'animation', 'comedy', 'drama', 'family', 'scifi']","['action', 'animation', 'comedy', 'family', 'fantasy']","['action', 'animation', 'comedy', 'family', 'music']","['action', 'animation', 'comedy', 'family']","['action', 'animation', 'comedy', 'fantasy', 'scifi', 'horror', 'drama']","['action', 'animation', 'comedy', 'fantasy']","['action', 'animation', 'comedy', 'thriller', 'scifi', 'family', 'romance', 'fantasy']","['action', 'animation', 'family', 'fantasy', 'comedy']","['action', 'animation', 'family', 'scifi', 'comedy']",...,"['war', 'history', 'documentation']","['war', 'reality', 'animation', 'action', 'comedy', 'drama', 'family']","['war', 'romance', 'thriller', 'drama', 'history', 'european']","['war', 'thriller', 'drama']","['war', 'thriller']",['war'],"['western', 'action', 'scifi', 'thriller', 'drama', 'fantasy']","['western', 'crime', 'drama']",['western'],[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
35,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5830,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
frames = [clean,df_dum_genre,df_dum_prod]
combined_df = pd.concat(frames)
combined_df

,release_year,age_certification,runtime,genres,seasons,imdb_score,imdb_votes,tmdb_popularity,"['action', 'animation', 'comedy', 'drama', 'family', 'fantasy', 'music', 'romance']","['action', 'animation', 'comedy', 'drama', 'family', 'scifi']",...,"['US', 'IN']","['US', 'IT']","['US', 'JP']","['US', 'KR']","['US', 'MX']","['US', 'NG']",['US'],['XX'],"['ZA', 'US']",['ZA']
0,1945.0,TV-MA,51.0,['documentation'],1.0,NaN,NaN,0.600,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1969.0,TV-14,30.0,"['comedy', 'european']",4.0,8.8,73424.0,17.617,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,1972.0,TV-MA,43.0,['comedy'],1.0,8.1,2151.0,1.487,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,1989.0,TV-PG,24.0,['comedy'],9.0,8.9,308824.0,130.213,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,1982.0,TV-PG,51.0,"['scifi', 'action', 'crime', 'drama']",4.0,6.9,34115.0,50.267,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


df_dummies = df_shows()